In [1]:
using TensorDecomposition, Symbolics, LinearAlgebra, Combinatorics, InvertedIndices, Einsum, IterTools, HomotopyContinuation

In [2]:
function kronMat(A::Matrix, d)
    if d == 1
        B = A
    else
        n, r = size(A)
        B = zeros(eltype(A), (n^d, r))
        for i=1:r 
            B[:, i] = kron(ntuple(x->A[:, i], d)...)
        end;
    end;
    return B
end;

function changeBasis(T, M)
    Tsize = size(T)
    n1 = Tsize[1]
    d = length(Tsize)
    
    n2 = size(M)[1]
    
    That = reshape(T, n1^d)
    return reshape(foldl(kron, repeat([M], d))*That, repeat([n2], d)...)
end;

function e(j, n)
    e = zeros(n)
    e[j] = 1
    return e
end;

function essentialVar(T, tol=1e-10)
    Tsize = size(T)
    n = Tsize[1]
    d = length(Tsize)
    
    That = reshape(T, (n, n^(d-1)))
    U, s, _ = svd(That)
    inds = (abs.(s) .> tol)
    if n == count(i->(i== 1), inds)
        return T, I
    else
        U = U[:, inds]
        return changeBasis(T, U'), U
    end
    
end;

function projectivize(L, A, d)
    return permutedims((Complex.(L) .^ (1/d))) .* A;
end;

function deprojectivize(A, d)
    B = A[1, :]
    return B .^ d, A ./ permutedims(B)
end;

function preprocess(T, tol=1e-10)
    T, U = essentialVar(T)
    
    Tsize = size(T)
    n = Tsize[1]
    d = length(Tsize)
    
    M = randn(n, n)
    M ./= sqrt(n)
    T = changeBasis(T, M)
    return T, U, M
end;

function multMon(x, j)
    c = copy(reverse(x))
    for i=1:length(c)
        if c[i] == 0
            c[i] = j
            break
        end
    end;
    return sort(reverse(c))
end;

function obtainDecomp(Ms, T)
    Ahat = zeros(ComplexF64, n, r)
    Ahat[1, :] = ones(r)
    for j=1:length(Ms)
        wj, Vj = eigen(Ms[j])
        argsort = sortperm(Vj[1, :], by=x -> (real(x), imag(x)), rev=true);
        Ahat[j+1, :] = wj[argsort]
    end
    Lhat = kronMat(Ahat, d)\reshape(T, n^d)
    return Lhat, Ahat
end;

function postprocess(Ahat, U, LT)
    return U*inv(LT)*Ahat
end;

function processLinEqs(linEqs, vars)
    A = []
    b = []
    
    z = zeros(length(vars))
    
    for linEq in linEqs
        coeffDict = HomotopyContinuation.to_dict(linEq, hs)
        push!(A, HomotopyContinuation.to_number.(sum([item.first * item.second for item in coeffDict])))
        push!(b, HomotopyContinuation.to_number(coeffDict[z]))
    end
    return Complex.(permutedims(reduce(hcat, A))), -Complex.(b)
    
end;

function hankelNew(T)
    Tsize = size(T)
    n = Tsize[1]
    d = length(Tsize)
    r = rank(catMat(T, Int(floor(d/2))))

    if r <= binomial(n-1+Int(floor((d-1)/2)), n-1)

        # Regime where Jennrich might work

        Tcat = catMat(T, Int(floor(d/2)))
        H0 = Tcat[1:r, 1:r];
        H0_inv = inv(H0)

        D = Dict()
        first_r = []
        for (i, ind) in enumerate(with_replacement_combinations(0:n-1, d))
            if i <= r
                push!(first_r, ind)
            end
            D[Tuple(x for x in ind)] = i
        end

        Hs = []
        for i=1:n-1
            hankInds = []
            multMap = map(x -> multMon(x, i), first_r)
            for ind in multMap
                push!(hankInds, D[Tuple(x for x in ind)])
            end
            push!(Hs, Tcat[1:r, hankInds])
        end

        Ms = []
        for H in Hs
            push!(Ms, H*H0_inv)
        end;

        eqMats = []
        for i=1:n-1
            for j=i+1:n-1
                push!(eqMats, Ms[i]*Ms[j]-Ms[j]*Ms[i])
            end
        end
        if all([all(abs.(eqMat) .<= tol) for eqMat in eqMats])
            Ahat_ = projectivize(obtainDecomp(Ms, T)..., d)
            Lhat, Ahat = deprojectivize(postprocess(Ahat_, U, LT), d)
        else
            error("Multiplication matrices do not commute")
        end

    elseif r <= binomial(n-1+Int(floor(d/2)), n-1)
        Thank = hankMat(T);
        H0 = Thank[1:r, 1:r];
        H0 = convert(Matrix{eltype(T)}, H0)
        H0_inv = inv(H0)

        D = Dict()
        first_r = []
        for (i, ind) in enumerate(with_replacement_combinations(0:n-1, d))
            if i <= r
                push!(first_r, ind)
            end
            D[Tuple(x for x in ind)] = i
        end

        Hs = []
        for i=1:n-1
            hankInds = []
            multMap = map(x -> multMon(x, i), first_r)
            for ind in multMap
                push!(hankInds, D[Tuple(x for x in ind)])
            end
            push!(Hs, Thank[1:r, hankInds])
        end

        hs = collect(Set(vcat(HomotopyContinuation.variables.(Hs)...)))

        eqMats = []
        for i=1:n-1
            for j=i+1:n-1
                push!(eqMats, Hs[i]*H0_inv*Hs[j]-Hs[j]*H0_inv*Hs[i])
            end
        end

        gam = binomial(n-1+Int(floor((d-1)/2)), n-1)
        linInds = 1:gam
        quadInds = gam+1:r;

        linEqs = []
        quadEqs = []
        for eqMat in eqMats
            append!(linEqs, HomotopyContinuation.expand.(eqMat[linInds, quadInds]))
            append!(quadEqs, HomotopyContinuation.expand.(eqMat[quadInds, quadInds]))
        end

        A, b = processLinEqs(linEqs, hs);

        if rank(A) == length(hs)
            sol = A\b
        else
            F = System(vcat(linEqs, quadEqs), hs)
            result = solve(F)
            sols = solutions(result, only_nonsingular=false)
            if length(sols) == 0
                error("There are no solutions h such that the multiplication matrices commute.  The input r is likely not the correct rank.")
            end
            sol = sols[1]
        end

        Ms = []
        for H in Hs
            push!(Ms, evaluate.(H, hs => sol)*H0_inv)
        end
        eqMatsEval = []
        for i=1:n-1
            for j=i+1:n-1
                push!(eqMatsEval, Ms[i]*Ms[j]-Ms[j]*Ms[i])
            end
        end

        if all([all(abs.(eqMat) .<= tol) for eqMat in eqMatsEval])
            Ahat_ = projectivize(obtainDecomp(Ms, T)..., d)
            Lhat, Ahat = deprojectivize(postprocess(Ahat_, U, LT), d)
        else
            error("Multiplication matrices do not commute")
        end

    end;
    return Lhat, Ahat
end;

In [296]:
n_ = 3
d_ = 4;

In [297]:
r_ = 8;

In [442]:
T_ = zeros(repeat([d_], d_)...)
for perm in Combinatorics.permutations(1:d_)
    T_[perm...] = 1
end
# LT = [[1, 1, 1, 1];; [1, 1, 1, 0];; [1, 1, 0, 0];; [1, 0, 0, 0]]
LT = randn(4, 4) /2;
# T_ = changeBasis(T_, LT)

# T_, A, L = randomRankedTensor(n_+1, d_, r_, real=true)
# A_ = projectivize(L, A_, d_)

# T, U, LT = preprocess(T_);
# T = T_;

In [443]:
P = zeros(d, r_)
P[1, :] = ones(r_)
sigs = product(repeat([[-1, 1]], d-1)...)
for (i, sig) in enumerate(sigs)
    P[2:end, i] .= sig
end

L_ = prod.(eachcol(P)) ./ (r_);

A_ = LT*projectivize(L_, P, d)

L, A = deprojectivize(A_, d)

T = real(makeRankedTensor(L, A, d));

In [444]:
rank(catMat(T, 2)[1:8, 1:8])

6

In [459]:
svdvals(catMat(changeBasis(makeRankedTensor(L_, P, d+2), LT), 3)[[(1:7)..., 11], [(1:7)..., 11]])

8-element Vector{Float64}:
 28.092182085929284
  2.823133286709094
  1.8791132511121262
  0.7188731422828085
  0.6168887503934739
  0.011954622284875585
  0.00023374421757807855
  2.0964520979241543e-5

In [457]:
rank(catMat(makeRankedTensor(L_, P, d+2), 3)[[(1:7)..., 11], [(1:7)..., 11]])

4

In [458]:
catMat(makeRankedTensor(L_, P, d+2), 3)[[(1:7)..., 11], [(1:7)..., 11]]

8×8 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [455]:
catMat(changeBasis(makeRankedTensor(L_, P, d+2), I), 3)[1:, 1:8]

LoadError: syntax: missing last argument in "1:" range expression 

In [317]:
n = n_+1
d = d_
r = 6;

In [318]:
ordering = Dict()
for (i, ind) in enumerate(with_replacement_combinations(1:n, d))
    ordering[ind] = i
end

function convertInds(ind, ordering, d)
    ind_ = []
    for i=1:length(ind)
        if !(ind[i]==0)
            append!(ind_, repeat([i+1], ind[i]))
        end
    end
    return ordering[prepend!(ind_, repeat([1], d-sum(ind)))]
end;

function cofactor(A::AbstractMatrix)
    ax = axes(A)
    out = similar(A, eltype(A), ax)
    for col in ax[1]
        for row in ax[2]
            out[col, row] = (-1)^(col + row) * det(A[Not(col), Not(row)])
        end
    end
    return out
end;

In [319]:
c = Int(floor(n_-(-1+sqrt(4*n_*(n_+3)-8*r+9))/2))
c_ = Int(r-n_-1-c*n_+c*(c-1)/2)

B = Set()
for i=1:c
    ei = zeros(Int8, n-1)
    ei[i] = 1
    for j=1:n-1
        ej = zeros(Int8, n-1)
        ej[j] = 1
        push!(B, ei+ej)
    end
end
ei = zeros(Int8, n-1)
ei[c+1] = 1
for j=c+1:c+c_
    ej = zeros(Int8, n-1)
    ej[j]=1
    push!(B, ei+ej)
end
B

Set{Any} with 2 elements:
  Int8[2, 0, 0]
  Int8[1, 1, 0]

In [323]:
Thank = hankMat(T);
H0 = Thank[1:r, 1:r];
H0 = convert(Matrix{eltype(T)}, H0)
H0_inv = inv(H0)

D = Dict()
first_r = []
for (i, ind) in enumerate(with_replacement_combinations(0:n-1, d))
    if i <= r
        push!(first_r, ind)
    end
    D[Tuple(x for x in ind)] = i
end

Hs = []
for i=1:n-1
    hankInds = []
    multMap = map(x -> multMon(x, i), first_r)
    for ind in multMap
        push!(hankInds, D[Tuple(x for x in ind)])
    end
    push!(Hs, Thank[1:r, hankInds])
end

hs = collect(Set(vcat(HomotopyContinuation.variables.(Hs)...)))

V = Set()
for alpha in B
    for alpha_ in B
        for i=1:n-1
            ei = zeros(Int8, n-1)
            ei[i] = 1
            push!(V, alpha+alpha_+ei)
        end
    end
end

E = Set()
for alpha in B
    for k=1:n-1
        beta = zeros(Int8, n-1)
        beta[k] = 1
        for j=1:n-1
            ej = zeros(Int8, n-1)
            ej[j] = 1
            if !(beta+ej in B)
                for i=1:n-1
                    ei = zeros(Int8, n-1)
                    ei[i] = 1
                    if alpha+beta+ei+ej in V && i != j
                        push!(E, (alpha+ei, beta+ej))
                    end
                end
            end
        end
    end
end

linEqs = []
for (alpha, beta) in E
    indA = convertInds(alpha, ordering, d)
    indB = convertInds(beta, ordering, d)
    linEq = Thank[indA, indB]*det(H0)-(permutedims(Thank[1:r, indA])*cofactor(H0)*Thank[1:r, indB])[1]
    push!(linEqs, HomotopyContinuation.expand(linEq))
end

Aeqs, beqs = processLinEqs(linEqs, hs);

In [327]:
sol = Aeqs \ beqs

Ms = []
for H in Hs
    push!(Ms, evaluate.(H, hs => sol)*H0_inv)
end
eqMatsEval = []
for i=1:n-1
    for j=i+1:n-1
        push!(eqMatsEval, Ms[i]*Ms[j]-Ms[j]*Ms[i])
    end
end

In [329]:
eqMatsEval[1]

6×6 Matrix{Float64}:
   8.97409e-13   -4.89775e-13   2.36155e-13  …    4.78017e-14  -3.49942e-13
   1.2399e-12    -2.58726e-12   5.19584e-14       8.69527e-13  -3.91687e-13
  -1.50449e-9     6.29837e-9    7.49136e-10      -2.41712e-9   -6.40155e-11
  -3.76207e-9     1.57047e-8    1.84375e-9       -6.02987e-9   -1.48209e-10
  -0.902983      -3.04301      -1.52363           1.4894        0.463655
 -22.2513       101.622        13.3791       …  -39.4458       -1.4894